In [26]:
import json
import base64
import requests
from Crypto.Cipher import AES
import pymysql

class AESCipher:
    def __init__(self, key):
        self.key = key.encode('utf-8')
        self.mode = AES.MODE_ECB

    def pad(self, data):
        block_size = AES.block_size
        padding = block_size - len(data) % block_size
        return data + (chr(padding) * padding).encode('utf-8')

    def unpad(self, data):
        return data[:-ord(data[-1])]

    def encrypt(self, raw):
        raw = self.pad(raw.encode('utf-8'))
        cipher = AES.new(self.key, self.mode)
        encrypted = base64.b64encode(cipher.encrypt(raw))
        return encrypted.decode('utf-8')

    def decrypt(self, enc):
        enc_bytes = base64.b64decode(enc)
        cipher = AES.new(self.key, self.mode)
        decrypted = self.unpad(cipher.decrypt(enc_bytes))
        return decrypted.decode('utf-8')
    
api_url = "https://qyss.cjbdi.com/qyss/qyss/enterlit/queryNewAllCase"
# api_url = "https://qyss.cjbdi.com/qyss/qyss/enterlit/queryNewLaCase"
key = "wkjtC@ntdGscsoqc"
ticket = "1LyM07LLxWpTBFnTqHGMdp1vHqGTYBF9Ck289X7P8ionseKdd62lHH3gxvJPr0wx"
companies = ["五矿二十三冶建设集团有限公司", "中国一冶集团有限公司", "中国华冶科工集团有限公司",
             "长沙五矿商业管理有限公司"]
loginName = "zgwk"
passWord = "shesu@2022&0198$"

: 

In [25]:
aes_cipher = AESCipher(key)
pageNum = 1
pageSize = 100

headers = {
    "Content-Type": "application/json",
    "ticket": ticket
}
request_payload = {
    "loginName": loginName,
    "passWord": passWord,
    "company": companies[0],
    "pageNum": pageNum,
    "pageSize": pageSize,
    "flag": 1
}

encrypted_payload = aes_cipher.encrypt(json.dumps(request_payload))

response = requests.post(api_url, data=encrypted_payload, headers=headers)

assert response.status_code == 200, "Request failed with status code {}".format(response.status_code)


decrypted_response = aes_cipher.decrypt(response.text)

print decrypted_response
    
#         try:
#             decrypted_response = aes_cipher.decrypt(response.text)
#         except Exception as e:
#             print "Decryption failed for page " + str(pageNum)
#             print "Response text: " + response.text
#             break

#         response_data = json.loads(decrypted_response)

#         if "code" not in response_data or response_data["code"] != 1000:
#             print "Error in response: " + response_data.get('msg', 'No message')
#             print "Full response: " + response_data
#             break

#         data = response_data.get("data")
#         if not data:
#             print "No data field in response"
#             break

#         hashMap = data.get("hashMap")
#         if not hashMap:
#             print "No hashMap field in data"
#             break

#         for table_name in all_data.keys():
#             table_data = hashMap.get(table_name)
#             if table_data:
#                 all_data[table_name].extend(table_data)

#         if pageNum >= data.get("pages", 0):
#             break

#         pageNum += 1


decrypt: type of enc: <type 'unicode'>


UnicodeEncodeError: 'ascii' codec can't encode characters in position 8-9: ordinal not in range(128)

In [19]:
len('我是')

6